# Data Analisys
------------------------------------------------------

### Data import

In [1]:
# Importing libraries
from importlib import reload  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime as dt
import sys
sys.path.append("src/")  # path contains python_file.py
import analysis_functions as af

# Importing data 
# * because an error we should use encoding='latin-1'

df = pd.read_csv('output/data_processed.csv', encoding='latin-1')
print(df.shape)
df.head()

(1425, 17)


,Unnamed: 0,Case Number,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,moment_day,Fatal_b,Species,shark_size,date
0,0,2018.06.25,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18,Afternoon,False,White,NaN,2018-06-25
1,6,2018.06.03.a,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,M,18,FATAL,Y,Afternoon,Afternoon,True,Tiger,NaN,2018-06-03
2,8,2018.05.26.b,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,M,15,Lower left leg bitten,N,17,Afternoon,False,Bull,1.8,2018-05-26
3,10,2018.05.24,Provoked,AUSTRALIA,Queensland,Cairns Aquarium,Feeding sharks,M,32,Minor bite to hand by captive shark. PROVOKED ...,N,Morning,Morning,False,Grey,NaN,2018-05-24
4,13,2018.05.13.a,Invalid,ENGLAND,Cornwall,Off Land's End,Fishing,M,21,Injured by teeth of a dead porbeagle shark he ...,N,8,Morning,False,Unidentified,NaN,2018-05-13


### Index creation

The index are useful to facilitate the data analysis

In [19]:
df['date_dt'] = pd.to_datetime(df['date'])

In [24]:
df['Year'] = df['date_dt'].dt.year
df['Month'] = df['date_dt'].dt.month
df=df.set_index(['Year', 'Month'])

In [28]:
df.Country.value_counts()

USA             676
AUSTRALIA       307
SOUTH AFRICA    216
BAHAMAS          25
NEW ZEALAND      20
               ... 
COLUMBIA          1
SCOTLAND          1
MALDIVES          1
SEYCHELLES        1
ARGENTINA         1
Name: Country, Length: 63, dtype: int64